In [ ]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [13]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
from config import engine_key

engine = create_engine(f'postgresql+psycopg2://{engine_key}')
conn = engine.connect()
df = pd.read_sql("SELECT * FROM Players", conn)
df.head()


,id,name,age,nationality,overall,potential,club,value,wage,preferredfoot,...,skill,movement,power,mentality,defending,goalkeeping,allstats,daysjoined,attackingworkrate,defensiveworkrate
0,158023,L. Messi,31,Argentina,94.0,94.0,FC Barcelona,110500000.0,565000.0,0,...,467.0,458.0,378.0,429.0,87.0,54.0,2298.0,5819.0,1,1
1,20801,Cristiano Ronaldo,33,Portugal,94.0,94.0,Juventus,77000000.0,405000.0,1,...,416.0,433.0,450.0,449.0,82.0,58.0,2323.0,697.0,2,0
2,190871,Neymar Jr,26,Brazil,92.0,93.0,Paris Saint-Germain,118500000.0,290000.0,1,...,444.0,458.0,353.0,443.0,84.0,59.0,2237.0,1038.0,2,1
3,193080,De Gea,27,Spain,91.0,93.0,Manchester United,72000000.0,260000.0,1,...,151.0,308.0,217.0,256.0,49.0,444.0,1539.0,3263.0,1,1
4,192985,K. De Bruyne,27,Belgium,91.0,92.0,Manchester City,102000000.0,355000.0,1,...,436.0,401.0,410.0,485.0,177.0,56.0,2369.0,1742.0,2,2


In [14]:
# Clean and convert Wage column
df['wage'] = df['wage'].replace('[\€,K]', '', regex=True)  
df['wage'] = pd.to_numeric(df['wage'], errors='coerce')   

In [15]:
# Check for missing values
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 id                      0
name                    0
age                     0
nationality             0
overall                 0
                     ... 
goalkeeping             0
allstats                0
daysjoined           1264
attackingworkrate       0
defensiveworkrate       0
Length: 64, dtype: int64


In [ ]:
#Preparing the data
df2 = df.drop(['id','value','potential'],axis=1)
df2 = df2[df2['position'] != 'GK']
df2 = df2.select_dtypes(include=['number'])
df2.head()

,age,overall,wage,preferredfoot,internationalreputation,weakfoot,skillmoves,height,weight,crossing,...,attacking,skill,movement,power,mentality,defending,goalkeeping,allstats,attackingworkrate,defensiveworkrate
0,31,94.0,565000.0,0,5.0,4.0,4.0,67.0,159.0,84.0,...,425.0,467.0,458.0,378.0,429.0,87.0,54.0,2298.0,1,1
1,33,94.0,405000.0,1,5.0,4.0,5.0,74.0,183.0,84.0,...,435.0,416.0,433.0,450.0,449.0,82.0,58.0,2323.0,2,0
2,26,92.0,290000.0,1,5.0,5.0,5.0,69.0,150.0,79.0,...,396.0,444.0,458.0,353.0,443.0,84.0,59.0,2237.0,2,1
4,27,91.0,355000.0,1,4.0,5.0,4.0,71.0,154.0,93.0,...,404.0,436.0,401.0,410.0,485.0,177.0,56.0,2369.0,2,2
5,27,91.0,340000.0,1,4.0,4.0,4.0,68.0,163.0,81.0,...,395.0,434.0,461.0,367.0,448.0,83.0,45.0,2233.0,2,1


In [19]:
# Splitting the data
y = df2['wage']
X = df2.drop(columns='wage')
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)

In [20]:
# Scaling the data
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression

In [21]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [22]:
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)

Model Coefficients: [-3.45460396e+03  9.51723348e+03  1.45415687e+02  1.11915139e+04
  1.12147749e+02 -3.95982948e+02  6.08223146e+02  2.93522877e+02
 -2.82354272e+14 -3.26528947e+14 -2.30807406e+14 -1.94743022e+14
 -2.94578833e+14  5.02154460e+14  6.08713791e+14  6.03725998e+14
  4.96865611e+14  4.01840759e+14  2.11275342e+15  2.07105679e+15
  2.21957774e+15  1.57469229e+15  2.19268222e+15  5.25625894e+14
  4.60372694e+14  4.44944785e+14  4.99405472e+14  6.26565424e+14
 -2.89541412e+14 -3.77772971e+14 -2.95436520e+14 -2.59172794e+14
 -2.50766691e+14 -2.03253573e+14  9.09967657e+14  1.01763125e+15
  1.01772569e+15  2.34123270e+15  2.32731493e+15  2.37406992e+15
  2.32641746e+15  2.33952820e+15  8.09968731e+14 -2.36131850e+15
 -7.98888455e+15 -1.68345451e+15  8.74860701e+14 -2.97942023e+15
 -6.02380391e+15  5.07277846e+14  4.51460938e+02  8.42755859e+02]
Model Intercept: 10183.55130874161


In [23]:
# Predict on the test data
y_pred = model.predict(X_test_scaled)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

# Compare a few predictions vs actual values
comparison_df = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print(comparison_df.head())

Mean Squared Error (MSE): 291792328.9156241
R-squared (R2): 0.5324068508766759
   Actual    Predicted
0  3000.0  4336.022948
1  6000.0  4221.191052
2  1000.0  -282.903120
3  7000.0  7929.034380
4  2000.0 -1597.278120


# Random Forest

In [11]:
# Initialize and train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predict and evaluate
rf_pred = rf_model.predict(X_test_scaled)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("Random Forest Results")
print("MSE:", rf_mse)
print("R² Score:", rf_r2)


Random Forest Results
MSE: 122017969.83678594
R² Score: 0.8045981720315679


# KNN 5

In [12]:
# Constructing training model
n_neighbors=5
knn5=KNeighborsRegressor(n_neighbors,weights='uniform')
knn5.fit(X_train_scaled,y_train)
y1_knn5=knn5.predict(X_train_scaled)
y1_knn5=list(y1_knn5)

c:\Users\mkori\anaconda3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\mkori\anaconda3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [13]:
# Evaluating model on training data
mse = mean_squared_error(y_train, y1_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_train_scaled,y_train)}")

The RMSE is: 11189.617440356862
The R-squared is 0.7431721574962407


In [14]:
# Using model on test data
y2_knn5=knn5.predict(X_test_scaled)
y2_knn5=list(y2_knn5)

In [15]:
# Evaluating model on test data
mse = mean_squared_error(y_test, y2_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_test_scaled,y_test)}")

The RMSE is: 16419.9884314175
The R-squared is 0.5682319313686602


# KNN 2

In [16]:
# Constructing model
n_neighbors=2
knn2=KNeighborsRegressor(n_neighbors,weights='uniform')
knn2.fit(X_train_scaled,y_train)
y1_knn2=knn2.predict(X_train_scaled)
y1_knn2=list(y1_knn2)

In [17]:
#Training
mse = mean_squared_error(y_train, y1_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_train_scaled,y_train)}")

The RMSE is: 8869.58205730527


The R-squared is 0.8386318062358111


In [18]:
# Testing
y2_knn2=knn2.predict(X_test_scaled)
y2_knn2=list(y2_knn2)

In [19]:
mse = mean_squared_error(y_test, y2_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_test_scaled,y_test)}")

The RMSE is: 16570.30287065873
The R-squared is 0.5602906299791762


# KNN 12

In [20]:
# Constructing Model
n_neighbors=12
knn12=KNeighborsRegressor(n_neighbors,weights='uniform')
knn12.fit(X_train_scaled,y_train)
y1_knn12=knn12.predict(X_train_scaled)
y1_knn12=list(y1_knn12)

In [21]:
# Training
mse = mean_squared_error(y_train, y1_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_train_scaled,y_train)}")

The RMSE is: 12382.265467316043
The R-squared is 0.6855063742706389


In [22]:
# Testing
y2_knn12=knn12.predict(X_test_scaled)
y2_knn12=list(y2_knn12)
mse = mean_squared_error(y_test, y2_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_test_scaled,y_test)}")

The RMSE is: 15441.599353734106
The R-squared is 0.6181529958901539


# SVC - Linear

In [23]:
# Constructing the model
from sklearn import svm
vect = svm.SVC(kernel='linear')
vect.fit(X_train_scaled,y_train)
y1_svc=vect.predict(X_train_scaled)
y1_svc=list(y1_svc)

In [24]:
# Training
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_train_scaled,y_train)}")

The RMSE is: 9313.070439157516
The R-squared is 0.3565149136577708


In [25]:
# Testing
y2_svc=vect.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_test_scaled,y_test)}")

The RMSE is: 16048.664614449812
The R-squared is 0.27840552416823605


# SVC - RBF

In [26]:
# Constructing model
rbf_svc =svm.SVC(kernel='rbf')
rbf_svc.fit(X_train_scaled,y_train)


SVC()

In [27]:
# Training
y1_svc=rbf_svc.predict(X_train_scaled)
y1_svc=list(y1_svc)
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_train_scaled,y_train)}")


The RMSE is: 20564.879910038027
The R-squared is 0.3770015698587127


In [28]:
# Testing
y2_svc=rbf_svc.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_test_scaled,y_test)}")

The RMSE is: 24479.073656512806
The R-squared is 0.2859384808537351


# XGBoost

In [30]:
!pip install xgboost
from xgboost import XGBRegressor
# Initialize and train XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Predict and evaluate
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print("XGBoost Results")
print("MSE:", xgb_mse)
print("R² Score:", xgb_r2)

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/124.9 MB 11.5 MB/s eta 0:00:11
   ---------------------------------------- 0.4/124.9 MB 5.5 MB/s eta 0:00:23
   ---------------------------------------- 0.6/124.9 MB 4.8 MB/s eta 0:00:26
   ---------------------------------------- 0.7/124.9 MB 4.0 MB/s eta 0:00:31
   ---------------------------------------- 1.1/124.9 MB 4.2 MB/s eta 0:00:30
   ---------------------------------------- 1.2/124.9 MB 3.9 MB/s eta 0:00:32
   ---------------------------------------- 1.4/124.9 MB 4.0 MB/s eta 0:00:32
   ---------------------------------------- 1.5/124.9 MB 3.9 MB/s eta 0:00:32
    --------------------------------------- 1.7/124.9 MB 3.5 MB/s eta 0:00:35
    --------------------------------------- 1.8/124.9 MB 3.4 MB/s eta 0:00:37
    --------------------------------------- 2.0/124.9 MB 3.3 MB/s eta 0:00:38
    --------------------------------------- 2.0/124.9 MB 3.1 MB/s eta 

# Neural Network

In [32]:
!pip install TensorFlow 
# Define a simple Neural Network
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = nn_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=1)

# Predict and evaluate
nn_pred = nn_model.predict(X_test_scaled).flatten()
nn_mse = mean_squared_error(y_test, nn_pred)
nn_r2 = r2_score(y_test, nn_pred)

print("Neural Network Results")
print("MSE:", nn_mse)
print("R² Score:", nn_r2)

c:\Users\mkori\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - loss: 557687232.0000 - mae: 10079.8291 - val_loss: 594592768.0000 - val_mae: 8225.9121
Epoch 2/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 383334816.0000 - mae: 7141.8291 - val_loss: 403874272.0000 - val_mae: 7645.6104
Epoch 3/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 293180832.0000 - mae: 7375.3237 - val_loss: 381457504.0000 - val_mae: 7789.8545
Epoch 4/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 309444768.0000 - mae: 7435.4272 - val_loss: 369751392.0000 - val_mae: 7614.4102
Epoch 5/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 267717376.0000 - mae: 7340.6396 - val_loss: 357680224.0000 - val_mae: 7535.2993
Epoch 6/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 276095776.0000 - mae: 7210.4180 - val_loss: 347746400.0000 - val_mae: 7385.3604
Epoch 7/50
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 244428768.0000 - mae: 7006.2383 - val_loss: 337954272.0000 - val_mae: 7247.7554
Epoch 8/50
